In [1]:
import os
os.makedirs("sample_pipeline", exist_ok=True)

In [2]:
%%writefile sample_pipeline/script1.py
from azureml.core import Run

run = Run.get_context()
run.log("message", "running_script_1")
run.complete()

Writing sample_pipeline/script1.py


In [3]:
%%writefile sample_pipeline/script2.py

from azureml.core import Run
run = Run.get_context()
run.log("message", "running_script_2")
run.complete()

Writing sample_pipeline/script2.py


In [19]:
from azureml.core.compute import AmlCompute, ComputeTarget
compute_name="aml-cluster"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                       min_nodes=0, max_nodes=2,
                                       vm_priority='lowpriority')
aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)
aml_cluster.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [25]:
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment, Workspace


step1 = PythonScriptStep(script_name="script1.py",
                 name="sample-pipeline-step1",
                 source_directory="sample_pipeline",
                        compute_target="aml-cluster")
step2 = PythonScriptStep(script_name="script2.py",
                 name="sample-pipeline-step2",
                 source_directory="sample_pipeline",
                        compute_target="aml-cluster")
ws = Workspace.from_config()
sample_pipeline = Pipeline(workspace=ws, steps=[step1, step2])
experiment = Experiment(workspace=ws, name="sample-pipeline")
run = experiment.submit(sample_pipeline)
run.wait_for_completion(show_output=True)

Created step sample-pipeline-step1 [0fbc1349][08ee608f-2394-4d78-bf72-4e59e6b2feff], (This step is eligible to reuse a previous run's output)
Created step sample-pipeline-step2 [2bbc7edc][57bda388-6d85-431f-aa02-f876e05df306], (This step is eligible to reuse a previous run's output)
Submitted PipelineRun 513f3ac3-49f8-4252-8e76-f2a0aade2a6e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/513f3ac3-49f8-4252-8e76-f2a0aade2a6e?wsid=/subscriptions/fcd1fe46-718c-472d-9814-211fa6d32599/resourcegroups/learn-mlops-rg/workspaces/learn-mlops-ws&tid=219401eb-53de-4dfb-8561-29e630707cb7
PipelineRunId: 513f3ac3-49f8-4252-8e76-f2a0aade2a6e
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/513f3ac3-49f8-4252-8e76-f2a0aade2a6e?wsid=/subscriptions/fcd1fe46-718c-472d-9814-211fa6d32599/resourcegroups/learn-mlops-rg/workspaces/learn-mlops-ws&tid=219401eb-53de-4dfb-8561-29e630707cb7
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 6823bfed-d4db-4ac5-ae

'Finished'